<a href="https://colab.research.google.com/github/bisi-leye/CofeedataAnalysis/blob/main/Woodcore_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Junior Data Analyst Assessment

## Task Guide
Write a small [one file] Python project that does the following.
1. Read data from a MYSQL database.
2. Perform simple analytics on the provided data following the analytics questions provide. The result headers should be in camel case.
3. Write result into an Excel File.

   
## Expected Delivery
1. Write each query result on a single sheet in the excel file. Specifically, the excel file should be a single workbook with multiple sheets representing the data from each query, the sheets should be properly named.
   
2. On a single run command, the application should process the steps above and generate
an excel file.
3. No Python framework is required.
4. Push to GitHub and share the URL with people@woodcoreapp.com


## Database Dictionary:
- Orders: Records of customer orders, including order_ids, timestamps, item_ids, quantities, customer names, and whether the order was for dine-in or takeout.
- Items: Details of menu items, including item IDs, SKUs, names, categories, sizes, and prices.
- Recipes: Information on how each menu item is made, listing required ingredients and quantities.
- Ingredients: A list of ingredients used in the coffee shop, including their IDs, names, weights, measurements, and prices.
- Inventory: Current stock levels of each ingredient.
- Staff: Information on coffee shop staff, including their IDs, names, positions, and salary rates.
- Shift: Details of work shifts, including shift IDs, days, start times, and end times.
- Rota: Staff work schedules, linking staff members to specific shifts.

In [1]:
# download SQL
!pip install mysql-connector-python
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 856.3 kB/s eta 0:00:00


In [2]:
import mysql.connector
import openpyxl
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from urllib.parse import quote_plus

In [ ]:
def main():
    config = {
        'user': 'root',
        'password': '@gopsenator!',
        'host': 'localhost',
        'database': 'coffeedata',
    }

    # URL encode password and create engine
    encoded_password = quote_plus(config['password'])
    engine_url = f"mysql+pymysql://{config['user']}:{encoded_password}@{config['host']}/{config['database']}"
    engine = create_engine(engine_url, pool_recycle=3600)

    # Context Manager for Connection Handling
    with engine.connect() as conn:
        df = pd.read_sql_table("ingredients", conn)
        print(df.head())

if __name__ == "__main__":
    main()

   ing_id        ing_name ing_weight ing_meas  ing_price
0  ING001  Espresso beans       1000    grams      12.00
1  ING002      Whole Milk       1000       ml       1.20
2  ING003         Cheddar        500    grams       7.45
3  ING004      Mozzarella        500    grams       5.00
4  ING005   Whipped cream        300       ml       1.35


## Analysis Questions

### Task 1:
Retrieve the total number of times each item has been ordered and the total revenue generated from these orders. Ensure that your final output includes all information identifying each item, including the number sold and revenue, and order the results to highlight the best-performing items first.

In [ ]:
def main():
    config = {
        'user': 'root',
        'password': '@gopsenator!',
        'host': 'localhost',
        'database': 'coffeedata',
    }
    encoded_password = quote_plus(config['password'])
    engine_url = f"mysql+pymysql://{config['user']}:{encoded_password}@{config['host']}/{config['database']}"
    engine = create_engine(engine_url, pool_recycle=3600)

    with engine.connect() as conn:
        # SQL query to retrieve total number of orders and total revenue for each item
        query = """
            SELECT
                items.item_id,
                items.item_name,
                COUNT(orders.row_id) AS total_orders,
                SUM(orders.quantity * items.item_price) AS total_revenue
            FROM
                orders
            JOIN
                items ON orders.item_id = items.item_id
            GROUP BY
                items.item_id, items.item_name
            ORDER BY
                total_orders DESC, total_revenue DESC;
        """
        df = pd.read_sql_query(query, conn)
        display(df)

        # Write to Excel
        with pd.ExcelWriter('tasks.xlsx') as writer:
            df.to_excel(writer, sheet_name='Item Performance', index=False)

if __name__ == "__main__":
    main()

,item_id,item_name,total_orders,total_revenue
0,It018,Cold Mocha,27,124.20
1,It005,Flat White,27,91.35
2,It009,Mocha,26,116.00
3,It011,White Mocha,23,103.50
4,It014,Hot Chocolate,22,110.40
5,It022,Lemonade,21,90.00
6,It001,Cappuccino,21,86.25
7,It020,Iced Tea,21,78.10
8,It012,White Mocha,20,122.20
9,It006,Caramel Macchiato,20,84.00


## Task 2:

Calculate the cost to produce each coffee item and utilize this information to determine profitability. Order the results by profitability, showing the most profitable items first and make sure to provide information identifying each item

In [ ]:
def main():
    config = {
        'user': 'root',
        'password': '@gopsenator!',
        'host': 'localhost',
        'database': 'coffeedata',
    }

    # URL encode password and create engine
    encoded_password = quote_plus(config['password'])
    engine_url = f"mysql+pymysql://{config['user']}:{encoded_password}@{config['host']}/{config['database']}"
    engine = create_engine(engine_url, pool_recycle=3600)

    # Context Manager for Connection Handling
    with engine.connect() as conn:
        # SQL query to calculate the cost to produce each coffee item and profitability
        query = """
            SELECT
                items.item_id,
                items.item_name,
                items.item_price,
                SUM(recipes.quantity * ingredients.ing_price) AS total_cost,
                items.item_price - SUM(recipes.quantity * ingredients.ing_price) AS profitability
            FROM
                items
            JOIN
                recipes ON items.item_id = recipes.recipe_id
            JOIN
                ingredients ON recipes.ing_id = ingredients.ing_id
            GROUP BY
                items.item_id, items.item_name, items.item_price
            ORDER BY
                profitability DESC;
        """
        df = pd.read_sql_query(query, conn)
        df.to_excel("tasks.xlsx", sheet_name="Profitability", index=False)

if __name__ == "__main__":
    main()

### Task 3:
Calculate the number of orders, sales, and profit for each hour the store is open

In [ ]:
def main():
    config = {
        'user': 'root',
        'password': '@gopsenator!',
        'host': 'localhost',
        'database': 'coffeedata',
    }

    encoded_password = quote_plus(config['password'])
    engine_url = f"mysql+pymysql://{config['user']}:{encoded_password}@{config['host']}/{config['database']}"
    engine = create_engine(engine_url, pool_recycle=3600)

    with engine.connect() as conn:
        query = """
            SELECT HOUR(created_at) AS hour_of_day,
                   COUNT(*) AS num_orders,
                   SUM(quantity * item_price) AS total_sales
            FROM orders o
            JOIN items i ON o.item_id = i.item_id
            WHERE in_or_out = 'out'
            GROUP BY hour_of_day
            ORDER BY hour_of_day;
        """
        df = pd.read_sql_query(query, conn)
        display(df)

        with pd.ExcelWriter('tasks.xlsx') as writer:
            df.to_excel(writer, sheet_name='Hourly Performance', index=False)

if __name__ == "__main__":
    main()

,hour_of_day,num_orders,total_sales
0,7,29,134.50
1,8,24,100.95
2,9,24,96.55
3,10,10,44.75
4,11,11,44.85
5,12,32,138.05
6,13,32,137.10
7,14,15,64.60
8,15,12,47.10
9,16,11,60.60


### Task 4:
Calculate the total hours worked by each staff member and their corresponding daily salaries. Ensure that the final result includes information identifying the staff, hours worked, and salaries earned.

In [ ]:
def main():
    config = {
        'user': 'root',
        'password': '@gopsenator!',
        'host': 'localhost',
        'database': 'coffeedata',
    }

    encoded_password = quote_plus(config['password'])
    engine_url = f"mysql+pymysql://{config['user']}:{encoded_password}@{config['host']}/{config['database']}"
    engine = create_engine(engine_url, pool_recycle=3600)

    with engine.connect() as conn:
        query = """
            SELECT
                staff.staff_id,
                CONCAT(staff.first_name, ' ', staff.last_name) AS staff_name,
                DATE(rota.date) AS work_date,
                SUM(TIMESTAMPDIFF(HOUR, shift.start_time, shift.end_time)) AS total_hours_worked,
                SUM(TIMESTAMPDIFF(HOUR, shift.start_time, shift.end_time) * staff.sal_per_hour) AS daily_salary
            FROM
                rota
            JOIN
                shift ON rota.shift_id = shift.shift_id
            JOIN
                staff ON rota.staff_id = staff.staff_id
            GROUP BY
                staff.staff_id, work_date
            ORDER BY
                work_date, staff.staff_id;
        """
        df = pd.read_sql_query(query, conn)
        display(df)
        with pd.ExcelWriter('tasks.xlsx') as writer:
            df.to_excel(writer, sheet_name='Staff Daily Performance', index=False)

if __name__ == "__main__":
    main()

,staff_id,staff_name,work_date,total_hours_worked,daily_salary
0,ST001,Emma Johnson,2024-02-12,None,None
1,ST003,Olivia Williams,2024-02-12,None,None
2,ST002,Liam Smith,2024-02-13,None,None
3,ST004,Noah Brown,2024-02-13,None,None
4,ST001,Emma Johnson,2024-02-14,None,None
5,ST003,Olivia Williams,2024-02-14,None,None
6,ST002,Liam Smith,2024-02-15,None,None
7,ST004,Noah Brown,2024-02-15,None,None
8,ST001,Emma Johnson,2024-02-16,None,None
9,ST002,Liam Smith,2024-02-16,None,None


### Task 5:
Retrieve the aggregate profit for dine-in and takeout orders, grouped by their respective categories.

In [ ]:
def main():
    # Database Configuration (Replace with your credentials)
    config = {
        'user': 'your_username',
        'password': 'your_password',
        'host': 'your_hostname',
        'database': 'coffeedata'
    }

    # Create the database connection URL
    encoded_password = quote_plus(config['password'])
    engine_url = f"mysql+pymysql://{config['user']}:{encoded_password}@{config['host']}/{config['database']}"
    engine = create_engine(engine_url)

    # SQL Query
    query = """
        SELECT in_or_out AS order_type,
               SUM(quantity * item_price) AS total_profit
        FROM orders o
        JOIN items i ON o.item_id = i.item_id
        WHERE in_or_out IN ('dine-in', 'takeout')
        GROUP BY in_or_out;
    """

    # Execute the query and load results into a DataFrame
    with engine.connect() as conn:
        df = pd.read_sql_query(query, conn)
        display(df)



if __name__ == "__main__":
    main()

### Task 6:
Which Shift is the busiest?

In [ ]:
def main():
    config = {
        'user': 'root',
        'password': '@gopsenator!',
        'host': 'localhost',
        'database': 'coffeedata',
    }

    # URL encode password and create engine
    encoded_password = quote_plus(config['password'])
    engine_url = f"mysql+pymysql://{config['user']}:{encoded_password}@{config['host']}/{config['database']}"
    engine = create_engine(engine_url, pool_recycle=3600)

    # Context Manager for Connection Handling
    with engine.connect() as conn:
        # SQL query to determine the busiest shift
        query = """
            SELECT
                CASE
                    WHEN HOUR(created_at) BETWEEN 6 AND 14 THEN 'Morning Shift'
                    WHEN HOUR(created_at) BETWEEN 15 AND 22 THEN 'Afternoon Shift'
                    ELSE 'Night Shift'
                END AS shift,
                COUNT(*) AS num_orders
            FROM
                orders
            GROUP BY
                shift
            ORDER BY
                num_orders DESC
            LIMIT 1;
        """

        df = pd.read_sql_query(query, conn)
        display(df)

        with pd.ExcelWriter('tasks.xlsx') as writer:
            df.to_excel(writer, sheet_name='Busiest Shift', index=False)

if __name__ == "__main__":
    main()

,shift,num_orders
0,Morning Shift,468
